# **Exploring HuggingFace Datasets**

In [2]:
from __future__ import annotations
from operator import itemgetter
from itertools import zip_longest
import fastcore.all as fc
import pickle, gzip, math, os, time, shutil, logging, os

from pathlib import Path
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch
from torch import tensor, nn, optim
from torch.utils.data import DataLoader, default_collate
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from fastcore.test import  test_close
from datasets import load_dataset, load_dataset_builder

# Configs
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=3, linewidth=125, sci_mode=False)
np.set_printoptions(precision=3, linewidth=125)

In [3]:
logging.disable(logging.WARNING)

## **Accessing HuggingFace Datasets**

Alongside the hub and spaces, HuggingFace also has a very large repository of datasets which can be used for experimentation, practice or portfolio building.

In [5]:
name  = "fashion_mnist"
ds_builder = load_dataset_builder(name)
print(ds_builder.info.description) # This provides handy meta data for the dataset.

Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of
60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image,
associated with a label from 10 classes. We intend Fashion-MNIST to serve as a direct drop-in
replacement for the original MNIST dataset for benchmarking machine learning algorithms.
It shares the same image size and structure of training and testing splits.



In [6]:
# We can dive deeper into the dataset features
ds_builder.info.features

{'image': Image(mode=None, decode=True, id=None),
 'label': ClassLabel(names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)}

In [7]:
# We can also get the training and test splits
ds_builder.info.splits

{'train': SplitInfo(name='train', num_bytes=31296655, num_examples=60000, shard_lengths=None, dataset_name=None),
 'test': SplitInfo(name='test', num_bytes=5233818, num_examples=10000, shard_lengths=None, dataset_name=None)}

In [8]:
# Once we're sure about the dataset in question, we can download them directly into a local object.
dsd = load_dataset(name)
dsd

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

The fact that we're getting everything organized and split by training and test sets, plus feature labels, is really cool. This functionality is similar to what was done in the previous notebook i.e. [06_mini_batch_training](https://github.com/bachaudhry/FastAI-22-23/blob/main/course_part_2/05_minibatch_training.ipynb).

The key difference is that we're now working with dictionaries instead of tuples.

In [9]:
train, test = dsd['train'], dsd['test']
train[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'label': 9}

In [13]:
# Storing labels as variables to reduce repetition.
#x, y = 'image', 'label'
# This is much cleaner.
x, y = ds_builder.info.features
img = train[0][x]
img

In [16]:
# Grabbing the first 5 images and labels
xb = train[:5][x]
yb = train[:5][y]
yb

[9, 0, 0, 3, 0]

In [19]:
# Get class mappings from features
feat = train.features[y]
feat

ClassLabel(names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)

In [27]:
# Direct class mappings on samples
feat.int2str(yb), train['label'][:5]

(['Ankle boot',
  'T - shirt / top',
  'T - shirt / top',
  'Dress',
  'T - shirt / top'],
 [9, 0, 0, 3, 0])

Now that we have our datasets, moving onto model training should be pretty straight forward.

Let's move onto the collate function which will be slighlty different as compared to the previous NB. This approach allows us to reduce the amount of abstraction which was typically necessary for previous iterations dataloaders. So `collate_fn()` will:

1. Loop through the each batch.
2. Convert items to Tensors.
3. Stack them up.

...in one go.

In [28]:
def collate_fn(b):
    # This will return a dictionary instead of a tuple in order to align with HuggingFace's approach.
    return {x: torch.stack([TF.to_tensor(o[x]) for o in b]),
            y: tensor([o[y] for o in b])} # This stacks on integers so we don't need to call torch.stack() explicitly

In [33]:
# Creating DataLoader
dl = DataLoader(train, collate_fn=collate_fn, batch_size=16)
b = next(iter(dl))
# The batch has 16 images with each image being 28x28
b[x].shape, b[y]

(torch.Size([16, 1, 28, 28]),
 tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9]))

HuggingFace also allows us to move away from collation functions for simplified and flexible custom functionality. 

In [34]:
def transforms(b):
    # Within each batch, replace the PIL images with tensors
    b[x] = [TF.to_tensor(o) for o in b[x]]
    return b

HuggingFace's `with_transform()` applies the transformation on the fly when `__getitem__()` is called.

In [36]:
# Using with_transform() applies the custom transformation
tds = train.with_transform(transforms)
dl = DataLoader(tds, batch_size=16) # Now we don't need a collate function
b = next(iter(dl))

b[x].shape, b[y]

(torch.Size([16, 1, 28, 28]),
 tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9]))

However, as Jeremy pointed out, having `transforms()` return an object requires extra code that can be avoided. We can write a custom transformation function which runs inplace and can be stored as a callable object.

In [40]:
# Creating transformation which can run inplace
def _transformi(b): b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [41]:
# Now we can create an additional function which runs the inplace operation
def inplace(f):
    def _f(b):
        f(b)
        return b
    return _f

In [43]:
# Calling the inplace transformation using our "function generating function"
# func     = _func    (_func)
transformi = inplace(_transformi)

In [45]:
# Rerunning with_transform() which returns the same result as the more verbose version above.
r = train.with_transform(transformi)[0]
r[x].shape, r[y]

(torch.Size([784]), 9)

The fun part is that what we just implemented above i.e. using `transformi = inplace(_transformi)` where a function generates other functions, mirrors the functionality of Python **decorators**.

In [46]:
# Calling inplace as a decorator and redefining the transform function.
@inplace
def transformi(b): b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [47]:
# Our results will be the same as before
tdsf = train.with_transform(transformi)
r = tdsf[0]

r[x].shape, r[y]

(torch.Size([784]), 9)

Moving on, we can also play around with the dynamic and highly flexible design of Python to add to the functionality of libraries like HuggingFace.

The `itemgetter()` function is a great example of this concept at work.

In [50]:
d = dict(a=1, b=2, c=3)
# Apply itemgetter to the dict above
ig = itemgetter('a', 'c')
ig(d)

(1, 3)

In [51]:
# This can be extended to classes as well.
class D:
    def __getitem__(self, k): return 1 if k=='a' else 2 if k=='b' else 3

In [58]:
d = D()
ig(d)

(1, 3)

In [59]:
list(tdsf.features)

['image', 'label']

...continuing on the thread of flexibility, we can also experiment with Python's `default_collate()` function, which works exactly as the name implies.

In [60]:
batch = dict(a=[1], b=[2]), dict(a=[3], b=[4])
batch

({'a': [1], 'b': [2]}, {'a': [3], 'b': [4]})

Let's see what the documentation for `default_collate` says.

In [63]:
default_collate?

Signature: default_collate(batch)
Docstring:
Take in a batch of data and put the elements within the batch into a tensor with an additional outer dimension - batch size.

The exact output type can be a :class:`torch.Tensor`, a `Sequence` of :class:`torch.Tensor`, a
Collection of :class:`torch.Tensor`, or left unchanged, depending on the input type.
This is used as the default function for collation when
`batch_size` or `batch_sampler` is defined in :class:`~torch.utils.data.DataLoader`.

Here is the general input type (based on the type of the element within the batch) to output type mapping:

    * :class:`torch.Tensor` -> :class:`torch.Tensor` (with an added outer dimension batch size)
    * NumPy Arrays -> :class:`torch.Tensor`
    * `float` -> :class:`torch.Tensor`
    * `int` -> :class:`torch.Tensor`
    * `str` -> `str` (unchanged)
    * `bytes` -> `bytes` (unchanged)
    * `Mapping[K, V_i]` -> `Mapping[K, default_collate([V_1, V_2, ...])]`
    * `NamedTuple[V1_i, V2_i, ...]` -> 

In [62]:
default_collate(batch)

{'a': [tensor([1, 3])], 'b': [tensor([2, 4])]}

We can now create a function called `collate_dict()` which will take a dataset and create an `itemgetter()` function for the features i.e. images and labels. This function will then call `default_collate()` in place to collate these items for us.

In [66]:
def collate_dict(ds):
    # Takes a dictionary and collates it into a tuple.
    get = itemgetter(*ds.features)
    def _f(b): return get(default_collate(b))
    return _f

In [65]:
# Calling collate_dict() inside the collate_fn for the DataLoader
dlf = DataLoader(tdsf, batch_size=4, collate_fn=collate_dict(tdsf))
xb, yby = next(iter(dlf))
xb.shape, yb

(torch.Size([4, 784]), [9, 0, 0, 3, 0])

This sort of flexibility allows us to easily move between HuggingFace and PyTorch, where data objects are usually stored as dictionaries for the prior and tuples in the case of the latter.|